<a href="https://colab.research.google.com/github/Siri319/Uber_Data/blob/main/uber_data_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())


  Preparing metadata (setup.py) ... done


In [6]:
import pandas as pd

# Replace 'your_file.csv' with your actual filename
df = pd.read_csv('uber-data-cleaned.csv')



In [ ]:
# Example SQL query
query = "SELECT * FROM df ;"
result = pysqldf(query)
print(result)


      Request id Pickup point  Driver id             Status Request date  \
0            619      Airport        1.0          Completed   11-07-2016   
1            867      Airport        1.0          Completed   11-07-2016   
2           1807         City        1.0          Completed   12-07-2016   
3           2532      Airport        1.0          Completed   12-07-2016   
4           3112         City        1.0          Completed   13-07-2016   
...          ...          ...        ...                ...          ...   
6740        6745         City        NaN  No Cars Available   15-07-2016   
6741        6752      Airport        NaN  No Cars Available   15-07-2016   
6742        6751         City        NaN  No Cars Available   15-07-2016   
6743        6754         City        NaN  No Cars Available   15-07-2016   
6744        6753      Airport        NaN  No Cars Available   15-07-2016   

     Request time  Drop  date Drop time  Trip Duration (mins)  
0        11:51:00  11-0

In [ ]:
# Total number of requests
query = "select count(`Request id`) from df ;"
result = pysqldf(query)
print(result)


   count(`Request id`)
0                 6745


In [ ]:
# Number of completed trips
query = "select count(`Request id`) from df where `Status` = 'Completed'"
result = pysqldf(query)
print("The number of completed trips -")
print(result)

The number of completed trips -
   count(`Request id`)
0                 2831


In [ ]:
# Number of cancelled trips
query = "select count(`Request id`) from df where `Status` = 'Cancelled'"
result = pysqldf(query)
print("The number of cancelled trips -")
print(result)


The number of cancelled trips -
   count(`Request id`)
0                 1264


In [ ]:
# Number of requests with “No Cars Available”
query = "select count(`Request id`) from df where `Status` = 'No Cars Available'"
result = pysqldf(query)
print("The number of requests with “No Cars Available” -")
print(result)

The number of requests with “No Cars Available” -
   count(`Request id`)
0                 2650


In [ ]:
# Number of requests from each pickup point (City vs Airport)
query = "select count(`Request id`) from df where `Pickup point` = 'City' ;"
result = pysqldf(query)
print(result)

query = "select count(`Request id`) from df where `Pickup point` = 'Airport' ;"
result = pysqldf(query)
print(result)

The number of requests from city -
   count(`Request id`)
0                 3507
The number of requests from airport 
   count(`Request id`)
0                 3238


In [ ]:
# Which pickup point has the most failures?
query = "select `Pickup point`, count(`Request id`) from df where `Status` = 'No Cars Available' group by `Pickup point`;"
result = pysqldf(query)
print(result)

  Pickup point  count(`Request id`)
0      Airport                 1713
1         City                  937


In [9]:
# Number of requests by hour of the day (peak hours)
query = "select strftime('%H', `Request time`), count(`Request id`) from df group by strftime('%H', `Request time`);"
result = pysqldf(query)
print(result)

   strftime('%H', `Request time`)  count(`Request id`)
0                              00                   99
1                              01                   85
2                              02                   99
3                              03                   92
4                              04                  203
5                              05                  445
6                              06                  398
7                              07                  406
8                              08                  423
9                              09                  431
10                             10                  243
11                             11                  171
12                             12                  184
13                             13                  160
14                             14                  136
15                             15                  171
16                             16                  159
17        

In [11]:
# Cancelled or failed requests by hour
query = "select strftime('%H', `Request time`), count(`Request id`) from df where `Status` = 'Cancelled' or `Status` = 'No Cars Available' group by strftime('%H', `Request time`);"
result = pysqldf(query)
print(result)


   strftime('%H', `Request time`)  count(`Request id`)
0                              00                   59
1                              01                   60
2                              02                   62
3                              03                   58
4                              04                  125
5                              05                  260
6                              06                  231
7                              07                  232
8                              08                  268
9                              09                  258
10                             10                  127
11                             11                   56
12                             12                   63
13                             13                   71
14                             14                   48
15                             15                   69
16                             16                   68
17        

In [12]:
# Trip completion rate by time of day
query = "select strftime('%H', `Request time`), count(`Request id`) from df where `Status` = 'Completed' group by strftime('%H', `Request time`);"
result = pysqldf(query)
print(result)

   strftime('%H', `Request time`)  count(`Request id`)
0                              00                   40
1                              01                   25
2                              02                   37
3                              03                   34
4                              04                   78
5                              05                  185
6                              06                  167
7                              07                  174
8                              08                  155
9                              09                  173
10                             10                  116
11                             11                  115
12                             12                  121
13                             13                   89
14                             14                   88
15                             15                  102
16                             16                   91
17        

In [17]:
# Average trip duration
query = "select avg(`Trip duration (mins)`) from df ;"
result = pysqldf(query)
print(result)
print("_________________________________________________________________")

# Minimum and maximum trip duration
query = "select min(`Trip duration (mins)`), max(`Trip duration (mins)`) from df ;"
result = pysqldf(query)
print(result)
print("_________________________________________________________________")

# Compare average trip duration between pickup points
query = "select `Pickup point`, avg(`Trip duration (mins)`) from df group by `Pickup point`;"
result = pysqldf(query)
print(result)

   avg(`Trip duration (mins)`)
0                     2.565642
_________________________________________________________________
   min(`Trip duration (mins)`)  max(`Trip duration (mins)`)
0                 -1413.033333                         83.0
_________________________________________________________________
  Pickup point  avg(`Trip duration (mins)`)
0      Airport                   -12.870774
1         City                    16.185406


In [21]:
# Number of repeated failures from a pickup point at a specific hour
query = "select `Pickup point`, strftime('%H', `Request time`), count(`Request id`) from df where `Status` = 'No Cars Available' group by `Pickup point`, strftime('%H', `Request time`);"
result = pysqldf(query)
print(result)
print("_____________________________________________________________________")
# Most common hours for cancelled trips
query = "select `Pickup point`, strftime('%H', `Request time`), count(`Request id`) from df where `Status` = 'Cancelled' group by `Pickup point`, strftime('%H', `Request time`);"
result = pysqldf(query)
print(result)
print("____________________________________________________________________")
# Times when No Cars Available is highest
query = "select `Pickup point`, strftime('%H', `Request time`), count(`Request id`) from df where `Status` = 'No Cars Available' group by `Pickup point`, strftime('%H', `Request time`);"
result = pysqldf(query)



   Pickup point strftime('%H', `Request time`)  count(`Request id`)
0       Airport                             00                   30
1       Airport                             01                   29
2       Airport                             02                   25
3       Airport                             03                   30
4       Airport                             04                   34
5       Airport                             05                    3
6       Airport                             06                    4
7       Airport                             07                    3
8       Airport                             08                    4
9       Airport                             09                    7
10      Airport                             10                   13
11      Airport                             11                   10
12      Airport                             12                   14
13      Airport                             13  

In [23]:
# Compare failure rates between City vs Airport
query = "select `Pickup point`, strftime('%H', `Request time`), count(`Request id`) from df where `Status` = 'No Cars Available' group by `Pickup point`, strftime('%H', `Request time`);"
result = pysqldf(query)
print(result)
print("_____________________________________________________________________")

# Failure rate by hour
query = "select strftime('%H', `Request time`), count(`Request id`) from df where `Status` = 'No Cars Available' group by strftime('%H', `Request time`);"
result = pysqldf(query)
print(result)

   Pickup point strftime('%H', `Request time`)  count(`Request id`)
0       Airport                             00                   30
1       Airport                             01                   29
2       Airport                             02                   25
3       Airport                             03                   30
4       Airport                             04                   34
5       Airport                             05                    3
6       Airport                             06                    4
7       Airport                             07                    3
8       Airport                             08                    4
9       Airport                             09                    7
10      Airport                             10                   13
11      Airport                             11                   10
12      Airport                             12                   14
13      Airport                             13  

In [24]:
# Requests per day
query = "select date(`Request date`), count(`Request id`) from df group by date(`Request date`);"
result = pysqldf(query)
print(result)
print("_____________________________________________________________________")

# Most active days
query = "select date(`Request date`), count(`Request id`) from df group by date(`Request date`) order by count(`Request id`) desc;"
result = pysqldf(query)
print(result)
print("_____________________________________________________________________")

# Failure trends across dates
query = "select date(`Request date`), count(`Request id`) from df where `Status` = 'No Cars Available' group by date(`Request date`);"
result = pysqldf(query)
print(result)



  date(`Request date`)  count(`Request id`)
0                 None                 6745
_____________________________________________________________________
  date(`Request date`)  count(`Request id`)
0                 None                 6745
_____________________________________________________________________
  date(`Request date`)  count(`Request id`)
0                 None                 2650
